In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import base64
#!pip install spotipy
import requests
from bs4 import BeautifulSoup
import time
from spotipy.oauth2 import SpotifyClientCredentials
#!pip install spotipy --upgrade

In [2]:
charts_df = pd.read_csv('../data/charts.csv')
lyrics_df = pd.read_csv('../data/billboard_24years_lyrics_spotify.csv')
spotify_qualities_df = pd.read_csv('../data/Hot100.csv')
song_attributes_df = pd.read_csv('../data/songAttributes_1999-2019.csv')

In [3]:
#spotify_qualities_df.info()
updated_song_attributes_df = song_attributes_df.drop(['Unnamed: 0', 'Popularity', 'Album'], axis=1)
updated_song_attributes_df = updated_song_attributes_df.rename(columns={'Name': 'song', 'Artist': 'artist'})
updated_song_attributes_df['artist'] = updated_song_attributes_df['artist'].str.lower()
updated_song_attributes_df['song'] = updated_song_attributes_df['song'].str.lower()
updated_song_attributes_df['artist'].value_counts()

artist
oakenfold         1797
willie nelson     1586
bing crosby       1434
cash cash         1406
elvis presley     1312
                  ... 
vitamin c           31
teairra mari        31
julianne hough      27
wham!               27
chris lake          21
Name: count, Length: 989, dtype: int64

In [7]:
updated_spotify_qualities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song              620 non-null    object 
 1   artist            620 non-null    object 
 2   Year              620 non-null    int64  
 3   Duration          620 non-null    int64  
 4   Time_Signature    620 non-null    int64  
 5   Danceability      620 non-null    float64
 6   Energy            620 non-null    float64
 7   Key               620 non-null    int64  
 8   Loudness          620 non-null    float64
 9   Mode              620 non-null    int64  
 10  Speechiness       620 non-null    float64
 11  Acousticness      620 non-null    float64
 12  Instrumentalness  620 non-null    float64
 13  Liveness          620 non-null    float64
 14  Valence           620 non-null    float64
 15  Tempo             620 non-null    float64
dtypes: float64(9), int64(5), object(2)
memory us

In [14]:
updated_spotify_qualities_df = spotify_qualities_df.drop(['Popularity', 'Album'], axis=1)
updated_spotify_qualities_df = updated_spotify_qualities_df.rename(columns={'Artist': 'main_artist', 'Track': 'song', 'Year': 'year'})
updated_spotify_qualities_df['main_artist'] = updated_spotify_qualities_df['main_artist'].str.lower()
updated_spotify_qualities_df['song'] = updated_spotify_qualities_df['song'].str.lower()
updated_spotify_qualities_df['year'].max()

2023

In [11]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['date'].dt.year
updated_charts_df = charts_df.drop(['date', 'last-week', 'rank'], axis=1)
updated_charts_df['artist'] = updated_charts_df['artist'].str.lower()
updated_charts_df['song'] = updated_charts_df['song'].str.lower()
updated_charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   song            330087 non-null  object
 1   artist          330087 non-null  object
 2   peak-rank       330087 non-null  int64 
 3   weeks-on-board  330087 non-null  int64 
 4   year            330087 non-null  int32 
dtypes: int32(1), int64(2), object(2)
memory usage: 11.3+ MB


In [12]:
featured_artists= ['featuring', '&', 'with', 'x']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

updated_charts_df['featured_artist'] = updated_charts_df['artist'].apply(extract_featured)
updated_charts_df['main_artist'] = updated_charts_df['artist'].apply(extract_main_artist)
updated_charts_df.tail(20)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
330067,beautiful delilah,chuck berry,81,1,1958,None,chuck berry
330068,blip blop,bill doggett,82,1,1958,None,bill doggett
330069,try the impossible,lee andrews and the hearts,83,1,1958,None,lee andrews and the hearts
330070,summertime blues,eddie cochran,84,1,1958,None,eddie cochran
330071,got a match?,the daddy-o's,85,1,1958,None,the daddy-o's
330072,to be loved,jackie wilson,86,1,1958,None,jackie wilson
330073,jealousy,kitty wells,87,1,1958,None,kitty wells
330074,just like in the movies,the upbeats,88,1,1958,None,the upbeats
330075,blue boy,jim reeves,89,1,1958,None,jim reeves
330076,stay,the ames brothers,90,1,1958,None,the ames brothers


In [29]:
range_updated_charts_df = updated_charts_df[
    (updated_charts_df['year'] >= 2000) &
    (updated_charts_df['year'] <= 2022)]
range_updated_charts_df.tail(20)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
114080,the chemicals between us,bush,67,14,2000,None,bush
114081,tha block is hot,lil' wayne featuring juvenile & b.g.,82,5,2000,juvenile & b.g.,lil' wayne
114082,guerrilla radio,rage against the machine,69,10,2000,None,rage against the machine
114083,"live, laugh, love",clay walker,84,5,2000,None,clay walker
114084,notorious b.i.g.,the notorious b.i.g. featuring puff daddy & li...,82,4,2000,puff daddy & lil' kim,the notorious b.i.g.
114085,she thinks my tractor's sexy,kenny chesney,74,7,2000,None,kenny chesney
114086,larger than life,backstreet boys,25,16,2000,None,backstreet boys
114087,there she goes,sixpence none the richer,32,17,2000,pence none the richer,si
114088,the greatest romance ever sold,prince,63,11,2000,None,prince
114089,the rockafeller skank,fatboy slim,77,24,2000,None,fatboy slim


In [28]:
filtered_artist_df = range_updated_charts_df[range_updated_charts_df['main_artist'].isin(updated_song_attributes_df['artist'])]
filtered_artist_df.head(30)

,song,artist,peak-rank,weeks-on-board,year,featured_artist,main_artist
0,easy on me,adele,1,3,2021,None,adele
4,bad habits,ed sheeran,2,18,2021,None,ed sheeran
5,way 2 sexy,drake featuring future & young thug,1,8,2021,future & young thug,drake
6,shivers,ed sheeran,7,7,2021,None,ed sheeran
21,knife talk,drake featuring 21 savage & project pat,4,8,2021,21 savage & project pat,drake
25,girls want girls,drake featuring lil baby,2,8,2021,lil baby,drake
36,ghost,justin bieber,32,5,2021,None,justin bieber
43,memory i don't mess with,lee brice,33,14,2021,None,lee brice
46,sharing locations,meek mill featuring lil baby & lil durk,22,9,2021,lil baby & lil durk,meek mill
51,fair trade,drake featuring travis scott,3,8,2021,travis scott,drake


In [25]:
updated_song_attributes_df['featured_artist'] = updated_song_attributes_df['song'].str.extract(
    r'(?:feat\.?|ft\.?|featuring|with)\s*[\.:]?\s*(.+?)(?:\)|$)')
updated_song_attributes_df = updated_song_attributes_df.query("featured_artist != 'you' and featured_artist != 'me' and featured_artist != 'out you'")
updated_song_attributes_df.head(20)

,Acousticness,artist,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,song,Speechiness,Tempo,TimeSignature,Valence,featured_artist
0,0.000728,collective soul,0.520,234947,0.904,False,0.010300,0.0634,-5.030,1,welcome all again,0.0309,106.022,4,0.365,NaN
1,0.018200,collective soul,0.581,239573,0.709,False,0.000664,0.1740,-4.909,1,fuzzy,0.0282,120.027,4,0.408,NaN
2,0.000473,collective soul,0.572,198400,0.918,False,0.000431,0.0977,-3.324,0,dig,0.0559,144.061,4,0.370,NaN
3,0.000970,collective soul,0.596,231453,0.661,False,0.000033,0.1130,-5.051,1,you,0.0254,111.975,4,0.183,NaN
4,0.000036,collective soul,0.520,222520,0.808,False,0.000010,0.0800,-4.553,0,my days,0.0318,92.721,4,0.666,NaN
5,0.010600,collective soul,0.353,263880,0.754,False,0.000073,0.1200,-4.570,1,understanding,0.0452,138.003,4,0.546,NaN
6,0.043600,collective soul,0.629,214973,0.787,False,0.000047,0.1000,-5.657,1,staring down,0.0277,100.440,4,0.476,NaN
7,0.000402,collective soul,0.646,206067,0.853,False,0.000090,0.1170,-4.685,1,she does,0.0278,127.963,4,0.699,NaN
8,0.000521,collective soul,0.645,216400,0.840,False,0.000011,0.0370,-4.995,1,lighten up,0.0697,140.025,4,0.795,NaN
9,0.000380,collective soul,0.601,212013,0.861,False,0.000002,0.1310,-4.690,1,love,0.0319,120.017,4,0.615,NaN


In [24]:
range_charts_qual_df = pd.merge(range_updated_charts_df, updated_spotify_qualities_df, on=['main_artist', 'year'], how='inner')
range_charts_qual_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17477 entries, 0 to 17476
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_x            17477 non-null  object 
 1   artist            17477 non-null  object 
 2   peak-rank         17477 non-null  int64  
 3   weeks-on-board    17477 non-null  int64  
 4   year              17477 non-null  int32  
 5   featured_artist   4217 non-null   object 
 6   main_artist       17477 non-null  object 
 7   song_y            17477 non-null  object 
 8   Duration          17477 non-null  int64  
 9   Time_Signature    17477 non-null  int64  
 10  Danceability      17477 non-null  float64
 11  Energy            17477 non-null  float64
 12  Key               17477 non-null  int64  
 13  Loudness          17477 non-null  float64
 14  Mode              17477 non-null  int64  
 15  Speechiness       17477 non-null  float64
 16  Acousticness      17477 non-null  float6

In [ ]:
chart_and_att_df = pd.merge(updated_charts_df, updated_song_attributes_df, on='song', how='inner')
grouped_chart_and_att_df = chart_and_att_df.groupby('song').first().reset_index()
grouped_chart_and_att_df.head(20)

In [ ]:
chart_att_and_qual_df = pd.merge(grouped_chart_and_att_df, updated_spotify_qualities_df, on=['year', 'song'], how='inner')
chart_att_and_qual_df.info()

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['featuring', '&', 'with']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

grouped_chart_and_att_df['featured_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_featured)
grouped_chart_and_att_df['main_artist'] = grouped_chart_and_att_df['artist_x'].apply(extract_main_artist)
grouped_chart_and_att_df['main_artist'].value_counts()

In [ ]:
grouped_lyrics_df = lyrics_df.groupby('song').first().reset_index()
grouped_lyrics_df

In [ ]:
charts_df.info()

In [ ]:
charts_df['artist'].value_counts()

In [ ]:
lyrics_df = lyrics_df.rename(columns={'ranking': 'rank'})
spotify_qualities_df = spotify_qualities_df.rename(columns={'Time_Signature': 'time_signature', 'Track': 'song', 'Year': 'year', 'Danceability': 'danceability', 'Energy': 'energy', 'Key': 'key', 'Loudness': 'loudness', 'Mode': 'mode', 'Speechiness': 'speechiness', 'Liveness': 'liveness', 'Valence': 'valence', 'Tempo': 'tempo'}) 
spotify_qualities_df.info()

In [ ]:
charts_df['datetime'] = pd.to_datetime(charts_df['date'])
charts_df['year'] = charts_df['datetime'].dt.year

In [ ]:
charts_df.info()

In [ ]:
charts_and_lyrics_df = pd.merge(charts_df, lyrics_df, on=['song'], how='inner')
pd.set_option('display.max_columns', None)
charts_and_lyrics_df.info()

In [ ]:
range_charts_and_lyrics_df = charts_and_lyrics_df[
    (charts_and_lyrics_df['year'] >= 1999) &
    (charts_and_lyrics_df['year'] <= 2023)]
no_repeats_range_charts_and_lyrics_df = range_charts_and_lyrics_df.groupby('song').first().reset_index()
no_repeats_range_charts_and_lyrics_df.head(10)

In [ ]:
#charts_and_lyrics_qualities_df = pd.merge(charts_and_lyrics_df, spotify_qualities_df, on=['year', 'song', 'time_signature', 'danceability', 'energy', 'loudness', 'speechiness', 'liveness', 'valence', 'tempo'], how='inner')
#pd.set_option('display.max_columns', None)
#charts_and_lyrics_qualities_df= charts_and_lyrics_qualities_df['song'].nunique()
#charts_and_lyrics_qualities_df.head(30)

In [ ]:
artists = sorted(charts_and_lyrics_df['artist'].astype(str).unique().tolist())
#artists

In [ ]:
#New columns for Main and Featured Artists
featured_artists= ['Featuring', '&', 'With']
def extract_featured(artist):
    for featured in featured_artists:
          if featured in artist:
            parts = artist.split(featured)
            if len(parts) > 1:
                return parts[1].strip()
    return None

def extract_main_artist(artist):
    for featured in featured_artists:
        if featured in artist:
            parts = artist.split(featured)
            return parts[0].strip()
    return artist.strip()

charts_and_lyrics_df['featured_artist'] = charts_and_lyrics_df['artist'].apply(extract_featured)
charts_and_lyrics_df['main_artist'] = charts_and_lyrics_df['artist'].apply(extract_main_artist)
charts_and_lyrics_df['main_artist'].head(20)

In [ ]:
spotify_qualities_df.dtypes

In [ ]:
#lyrics_df['time_signature'].value_counts()
spotify_qualities_df['time_signature'].value_counts()

In [ ]:
charts_and_lyrics_df[charts_and_lyrics_df['time_signature'] == 5]

In [ ]:
spotify_qualities_df['year'].max()
charts_and_lyrics_df['year'].max()

In [ ]:
charts_and_lyrics_df.info()

In [ ]:
updated_charts_and_lyrics_df = no_repeats_range_charts_and_lyrics_df.combine_first(spotify_qualities_df.set_index('song')).reset_index()
updated_charts_and_lyrics_df.head()